In [124]:
#5個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route.xlsx"
filename2=("fire_route.xlsx")
N_number=5
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([1])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 4, 2: 2, 3: 4, 5: 4}, 2: {1: 3, 2: 2, 3: 3, 5: 3}}
Q: {1: 6, 2: 2, 3: 4, 5: 6}
H: {1: 2, 2: 3, 3: 3, 5: 2}
b: {1: 18, 2: 8, 3: 12, 5: 18}


In [125]:
df = pd.read_excel(filename1)
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_p:
        A_p.append((u,v))
    if mode == "random":
        tau[u,v,fighterIndex] = rand_fightertraveltime[i]
    else:
        tau[u,v,fighterIndex] = time   
       
    
df = pd.read_excel(filename2)
df_num = len(df.index)
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_f:
        A_f.append((u,v))
    if mode == "random":
        lamb[u,v] = rand_firetraveltime[i]   #隨機產生資料用這行
    else:
        lamb[u,v] = time   #excel給定用這行
    
# print(tau)
# print(lamb)

A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點
    
for l in N-N_D:                          #定義A_f_NEIGHBOR
    connect = A_f.select('*',l)
    A_f_NEIGHBOR[l]=[]
    for temp in connect:
        A_f_NEIGHBOR[l].append(temp[0])
                      
A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)
model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}
epsilon=1e-3

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B",name="u")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")
        
#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")        
        
model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)
    
#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for k in K:
    for t in range(1,T_number):
        for j in N: 
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[k][j] <= t:
                    temp += u_bar[j,k,t-process[k][j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect),name="flow") #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12
        
#######################################################################################################################
    
for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0,name='test')
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])
            
for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)
    
for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)
    
for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)
            
for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+1]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k]]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])
                    
                    
model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
                   gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j),GRB.MINIMIZE)


model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)
                    

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5139 rows, 5183 columns and 17807 nonzeros
Model fingerprint: 0x9086d4ba
Variable types: 0 continuous, 5183 integer (5183 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-03, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 3987 rows and 3955 columns
Presolve time: 0.04s
Presolved: 1152 rows, 1228 columns, 3934 nonzeros
Variable types: 0 continuous, 1228 integer (1228 binary)
Found heuristic solution: objective 26.0050000

Root relaxation: objective 2.600083e+01, 339 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   26.00083    0  252   26.00500   26.00083  0.02%     -    0s
H    0     0        

In [126]:
EPS=1.e-2
print("x:")
step=1
for k in K:
    print()
    print("消防員%d的路徑" % k)
    temp = [elem for elem in x if elem[2]==k]
    for (i,j,k,t) in temp:
        if x[i,j,k,t].X >EPS:
            #print("x[%d,%d,%d,%d]" %(i,j,k,t) , x[i,j,k,t].X)
            if i != j:
                print("在時刻 %d 從node%d 移動到 node%d" %(t,i,j)," ,travel time:",tau[i,j,k])
            else:            
                if u_bar[i,k,t].X == 1:
                    print("在時刻 %d 對node%d進行保護"%(t,i)," ,processing time:",process[k][i])
                else:
                    print("在時刻 %d 在node%d idle" %(t,i))

print("w:")
for (i,k,t) in w:
    if w[i,k,t].X>EPS:
        print("w[%d,%d,%d]" %(i,k,t) , w[i,k,t].X)

print("u:")
for (i,t) in u:
    if u[i,t].X>EPS:
        print("u[%d,%d]" %(i,t) , u[i,t].X)

print("u_bar:")
for (i,k,t) in u_bar:
    if u_bar[i,k,t].X>EPS:
        print("u_bar[%d,%d,%d]" %(i,k,t) , u_bar[i,k,t].X)

print("v:")
for (i,t) in v:
    if v[i,t].X>EPS:
        print("v[%d,%d]" %(i,t) , v[i,t].X)

print("v_bar:")
for (i,t) in v_bar:
    if v_bar[i,t].X>EPS:
        print("v_bar[%d,%d]" %(i,t) , v_bar[i,t].X) 

x:

消防員1的路徑
在時刻 0 從node4 移動到 node3  ,travel time: 2
在時刻 2 對node3進行保護  ,processing time: 4
在時刻 6 從node3 移動到 node4  ,travel time: 2
在時刻 8 在node4 idle
在時刻 9 在node4 idle
在時刻 10 在node4 idle
在時刻 11 在node4 idle
在時刻 12 在node4 idle
在時刻 13 在node4 idle
在時刻 14 在node4 idle
在時刻 15 在node4 idle
在時刻 16 在node4 idle
在時刻 17 在node4 idle
在時刻 18 在node4 idle
在時刻 19 在node4 idle
在時刻 20 在node4 idle
在時刻 21 在node4 idle
在時刻 22 在node4 idle
在時刻 23 在node4 idle
在時刻 24 在node4 idle
在時刻 25 在node4 idle
在時刻 26 在node4 idle
在時刻 27 在node4 idle
在時刻 28 在node4 idle
在時刻 29 在node4 idle
在時刻 30 在node4 idle
在時刻 31 在node4 idle
在時刻 32 在node4 idle
在時刻 33 在node4 idle
在時刻 34 在node4 idle
在時刻 35 在node4 idle
在時刻 36 在node4 idle
在時刻 37 在node4 idle
在時刻 38 在node4 idle
在時刻 39 在node4 idle
在時刻 40 在node4 idle
在時刻 41 在node4 idle
在時刻 42 在node4 idle
在時刻 43 在node4 idle
在時刻 44 在node4 idle
在時刻 45 在node4 idle
在時刻 46 在node4 idle
在時刻 47 在node4 idle
在時刻 48 在node4 idle
在時刻 49 在node4 idle
在時刻 50 在node4 idle
在時刻 51 在node4 idle
在時刻 52 在node4 idle
在時刻 53 在node4 idl

In [81]:
#6個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_2.xlsx"
filename2=("fire_route_2.xlsx")
N_number=6
T_number =70                         #時間長度
K = set([1,2,3])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([6])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4,3:5}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 4, 2: 2, 3: 4, 5: 4, 6: 1}, 2: {1: 3, 2: 2, 3: 3, 5: 3, 6: 1}, 3: {1: 3, 2: 2, 3: 3, 5: 3, 6: 1}}
Q: {1: 6, 2: 2, 3: 4, 5: 6, 6: 3}
H: {1: 2, 2: 3, 3: 3, 5: 2, 6: 1}
b: {1: 18, 2: 8, 3: 12, 5: 18, 6: 50}


In [163]:
#7個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_3.xlsx"
filename2=("fire_route_3.xlsx")
N_number=7
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([3,5,7])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:5}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 4, 2: 2, 3: 4, 5: 4, 6: 1, 7: 3}, 2: {1: 3, 2: 2, 3: 3, 5: 3, 6: 1, 7: 2}}
Q: {1: 6, 2: 2, 3: 4, 5: 6, 6: 3, 7: 3}
H: {1: 2, 2: 3, 3: 3, 5: 2, 6: 1, 7: 3}
b: {1: 18, 2: 8, 3: 12, 5: 18, 6: 50, 7: 100}


In [253]:
#10個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_4.xlsx"
filename2=("fire_route_4.xlsx")
N_number=10
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([6,9])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:5}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 4, 2: 2, 3: 4, 5: 4, 6: 1, 7: 3, 8: 2, 9: 3, 10: 14}, 2: {1: 3, 2: 2, 3: 3, 5: 3, 6: 1, 7: 2, 8: 2, 9: 2, 10: 9}}
Q: {1: 6, 2: 2, 3: 4, 5: 6, 6: 3, 7: 3, 8: 3, 9: 2, 10: 7}
H: {1: 2, 2: 3, 3: 3, 5: 2, 6: 1, 7: 3, 8: 2, 9: 4, 10: 6}
b: {1: 18, 2: 8, 3: 12, 5: 18, 6: 50, 7: 100, 8: 10, 9: 20, 10: 15}


In [337]:
#15個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_5.xlsx"
filename2=("fire_route_5.xlsx")
N_number=15
T_number =70                         #時間長度
K = set([1,2])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([12,5,1])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7,11:3,12:3,13:3,14:6,15:6}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15,11:10,12:10,13:10,14:5,15:35}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6,11:7,12:5,13:2,14:2,15:5}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 4, 2: 2, 3: 4, 5: 4, 6: 1, 7: 3, 8: 2, 9: 3, 10: 14, 11: 7, 12: 5, 13: 2, 14: 4, 15: 10}, 2: {1: 3, 2: 2, 3: 3, 5: 3, 6: 1, 7: 3, 8: 2, 9: 2, 10: 11, 11: 6, 12: 4, 13: 2, 14: 3, 15: 8}}
Q: {1: 6, 2: 2, 3: 4, 5: 6, 6: 3, 7: 3, 8: 3, 9: 2, 10: 7, 11: 3, 12: 3, 13: 3, 14: 6, 15: 6}
H: {1: 2, 2: 3, 3: 3, 5: 2, 6: 1, 7: 3, 8: 2, 9: 4, 10: 6, 11: 7, 12: 5, 13: 2, 14: 2, 15: 5}
b: {1: 18, 2: 8, 3: 12, 5: 18, 6: 50, 7: 100, 8: 10, 9: 20, 10: 15, 11: 10, 12: 10, 13: 10, 14: 5, 15: 35}


In [430]:
#20個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="firefighters_route_6.xlsx"
filename2=("fire_route_6.xlsx")
N_number=20
T_number =70                         #時間長度
K = set([1,2,3])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([10,18])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4,3:2}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7,11:3,12:3,13:3,14:6,15:6,16:3,17:3,18:3,19:6,20:6}                  #q=單位時間火在i點燒的燃料量
    b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15,11:10,12:10,13:10,14:5,15:35,16:10,17:10,18:10,19:5,20:35}               #b=節點i的價值
    H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6,11:7,12:5,13:2,14:2,15:5,16:7,17:5,18:2,19:2,20:5}                  #h=火燒i點需要的時間
       
T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 4, 2: 2, 3: 4, 5: 4, 6: 1, 7: 3, 8: 2, 9: 3, 10: 14, 11: 7, 12: 5, 13: 2, 14: 4, 15: 10, 16: 7, 17: 5, 18: 2, 19: 4, 20: 10}, 2: {1: 3, 2: 2, 3: 3, 5: 3, 6: 1, 7: 3, 8: 2, 9: 2, 10: 11, 11: 6, 12: 4, 13: 2, 14: 3, 15: 8, 16: 6, 17: 4, 18: 2, 19: 3, 20: 8}, 3: {1: 6, 2: 3, 3: 6, 5: 6, 6: 2, 7: 5, 8: 3, 9: 4, 10: 21, 11: 11, 12: 8, 13: 3, 14: 6, 15: 15, 16: 11, 17: 8, 18: 3, 19: 6, 20: 15}}
Q: {1: 6, 2: 2, 3: 4, 5: 6, 6: 3, 7: 3, 8: 3, 9: 2, 10: 7, 11: 3, 12: 3, 13: 3, 14: 6, 15: 6, 16: 3, 17: 3, 18: 3, 19: 6, 20: 6}
H: {1: 2, 2: 3, 3: 3, 5: 2, 6: 1, 7: 3, 8: 2, 9: 4, 10: 6, 11: 7, 12: 5, 13: 2, 14: 2, 15: 5, 16: 7, 17: 5, 18: 2, 19: 2, 20: 5}
b: {1: 18, 2: 8, 3: 12, 5: 18, 6: 50, 7: 100, 8: 10, 9: 20, 10: 15, 11: 10, 12: 10, 13: 10, 14: 5, 15: 35, 16: 10, 17: 10, 18: 10, 19: 5, 20: 35}


In [1]:
#30個節點
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "not"
filename1="G30_firefighter_route.xlsx"
filename2=("G30_fire_route.xlsx")
filename3=("G30_nodeInformation.xlsx")
N_number=30
T_number = 150                         #時間長度
K = set([1])                        #K=消防車集合
N = set([i for i in range(1,N_number+1)])  #所有點的集合
N_D = set([4])                      #Depot
N_F = set([22])                      #起火點
rand_fightertraveltime=[]
rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3}                                  #各個消防車單位時間處理的燃料量

"""
if mode == "random":
    for i in range(1,N_number+1):
        if i != 14:
            Q[i] = random.randint(1,10)
            b[i] = random.randint(30,100)
            H[i] = random.randint(1,10)
    #travel time為隨機
    df = pd.read_excel(filename1)
    df_num = len(df.index)
    print(df_num)
   
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_fightertraveltime.append(temp)

    df = pd.read_excel(filename2)
    df_num = len(df.index)
    for i in range(df_num):
        temp=random.randint(1,10)
        for j in range(2):
            rand_firetraveltime.append(temp)
else:
    #H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    #Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    #b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
    
    #Q={1:6,2:2,3:4,5:6,6:3,7:3,8:3,9:2,10:7,11:3,12:3,13:3,14:6,15:6,16:3,17:3,18:3,19:6,20:6}                  #q=單位時間火在i點燒的燃料量
    #b={1:18,2:8,3:12,5:18,6:50,7:100,8:10,9:20,10:15,11:10,12:10,13:10,14:5,15:35,16:10,17:10,18:10,19:5,20:35}               #b=節點i的價值
    #H={1:2,2:3,3:3,5:2,6:1,7:3,8:2,9:4,10:6,11:7,12:5,13:2,14:2,15:5,16:7,17:5,18:2,19:2,20:5}                  #h=火燒i點需要的時間
    Q={}
    b={}
    H={}
    
    for i in range(1,N_number+1):
        if i not in N_D:
            Q[i] = random.randint(5,8)
            #Q[i] = 5
            b[i] = random.choice([5,10,15])
            #b[i]=30
            H[i] = random.randint(5,8)
            #H[i] = 5
"""

Q={}
b={}
H={}

df = pd.read_excel(filename3)
df_num = len(df.index)

for i in range(0,df_num):
    value = df.iloc[i]['value']
    h = df.iloc[i]['burning time']
    q = df.iloc[i]['quantity']
    Q[i + 1] = q
    b[i + 1] = value
    H[i + 1] = h

T = list([i for i in range(T_number+1)])    #時間集合
M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
for k in K:
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)
print(tau)



{1: {1: 17, 2: 16, 3: 14, 4: 12, 5: 9, 6: 10, 7: 19, 8: 16, 9: 10, 10: 16, 11: 19, 12: 17, 13: 12, 14: 14, 15: 14, 16: 14, 17: 17, 18: 14, 19: 19, 20: 14, 21: 14, 22: 16, 23: 12, 24: 16, 25: 14, 26: 12, 27: 19, 28: 10, 29: 12, 30: 12}}
Q: {1: 7, 2: 8, 3: 7, 4: 5, 5: 5, 6: 6, 7: 8, 8: 6, 9: 5, 10: 6, 11: 7, 12: 7, 13: 6, 14: 8, 15: 7, 16: 8, 17: 7, 18: 5, 19: 7, 20: 6, 21: 6, 22: 6, 23: 5, 24: 8, 25: 5, 26: 6, 27: 7, 28: 6, 29: 7, 30: 5}
H: {1: 7, 2: 6, 3: 6, 4: 7, 5: 5, 6: 5, 7: 7, 8: 8, 9: 6, 10: 8, 11: 8, 12: 7, 13: 6, 14: 5, 15: 6, 16: 5, 17: 7, 18: 8, 19: 8, 20: 7, 21: 7, 22: 8, 23: 7, 24: 6, 25: 8, 26: 6, 27: 8, 28: 5, 29: 5, 30: 7}
b: {1: 15, 2: 15, 3: 5, 4: 10, 5: 15, 6: 15, 7: 10, 8: 5, 9: 5, 10: 15, 11: 5, 12: 10, 13: 5, 14: 5, 15: 15, 16: 15, 17: 5, 18: 5, 19: 10, 20: 5, 21: 5, 22: 5, 23: 5, 24: 15, 25: 15, 26: 10, 27: 10, 28: 10, 29: 10, 30: 10}
{}


In [2]:
df = pd.read_excel(filename1)
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_p:
        A_p.append((u,v))
    if mode == "random":
        tau[u,v,fighterIndex] = rand_fightertraveltime[i]
    else:
        tau[u,v,fighterIndex] = time   
       
    
df = pd.read_excel(filename2)
df_num = len(df.index)
for i in range(0,df_num):
    u = df.iloc[i]['i']
    v = df.iloc[i]['j']
    time = df.iloc[i]['travel time']
    if (u,v) not in A_f:
        A_f.append((u,v))
    if mode == "random":
        lamb[u,v] = rand_firetraveltime[i]   #隨機產生資料用這行
    else:
        lamb[u,v] = time   #excel給定用這行
    
# print(tau)
# print(lamb)

A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點
    
for l in N-N_D:                          #定義A_f_NEIGHBOR
    connect = A_f.select('*',l)
    A_f_NEIGHBOR[l]=[]
    for temp in connect:
        A_f_NEIGHBOR[l].append(temp[0])
                      
A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)
model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}
epsilon=1e-3

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B",name="u")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")
        
#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")        
        
model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)
    
#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for k in K:
    for t in range(1,T_number):
        for j in N: 
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[k][j] <= t:
                    temp += u_bar[j,k,t-process[k][j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect),name="flow") #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12
        
#######################################################################################################################
    
for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0,name='test')
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])
            
for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)
    
for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)
    
for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)
            
for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+1]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k]]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])
                    
                    
model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
                   gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j),GRB.MINIMIZE)


model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)
                    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-29
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 54702 rows, 47716 columns and 196921 nonzeros
Model fingerprint: 0x82af5bca
Variable types: 0 continuous, 47716 integer (47716 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-03, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 42626 rows and 28716 columns
Presolve time: 0.74s
Presolved: 12076 rows, 19000 columns, 54710 nonzeros
Variable types: 0 continuous, 19000 integer (19000 binary)
Found heuristic solution: objective 260.0080000

Root relaxation: objective 1.363040e+02, 11865 iterations, 1.68 seconds (2.66 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/

In [3]:
EPS=1.e-2
print("x:")
step=1
for k in K:
    print()
    print("消防員%d的路徑" % k)
    temp = [elem for elem in x if elem[2]==k]
    for (i,j,k,t) in temp:
        if x[i,j,k,t].X >EPS:
            #print("x[%d,%d,%d,%d]" %(i,j,k,t) , x[i,j,k,t].X)
            if i != j:
                print("在時刻 %d 從node%d 移動到 node%d" %(t,i,j)," ,travel time:",tau[i,j,k])
            else:            
                if u_bar[i,k,t].X == 1:
                    print("在時刻 %d 對node%d進行保護"%(t,i)," ,processing time:",process[k][i])
                else:
                    print("在時刻 %d 在node%d idle" %(t,i))

print("w:")
for (i,k,t) in w:
    if w[i,k,t].X>EPS:
        print("w[%d,%d,%d]" %(i,k,t) , w[i,k,t].X)

print("u:")
for (i,t) in u:
    if u[i,t].X>EPS:
        print("u[%d,%d]" %(i,t) , u[i,t].X)

print("u_bar:")
for (i,k,t) in u_bar:
    if u_bar[i,k,t].X>EPS:
        print("u_bar[%d,%d,%d]" %(i,k,t) , u_bar[i,k,t].X)

print("v:")
for (i,t) in v:
    if v[i,t].X>EPS:
        print("v[%d,%d]" %(i,t) , v[i,t].X)

print("v_bar:")
for (i,t) in v_bar:
    if v_bar[i,t].X>EPS:
        print("v_bar[%d,%d]" %(i,t) , v_bar[i,t].X) 

x:

消防員1的路徑
在時刻 0 從node4 移動到 node28  ,travel time: 10.0
在時刻 10 從node28 移動到 node18  ,travel time: 16.0
在時刻 26 從node18 移動到 node15  ,travel time: 13.0
在時刻 39 對node15進行保護  ,processing time: 14
在時刻 53 從node15 移動到 node18  ,travel time: 13.0
在時刻 66 對node18進行保護  ,processing time: 14
在時刻 80 從node18 移動到 node28  ,travel time: 16.0
在時刻 96 對node28進行保護  ,processing time: 10
在時刻 106 從node28 移動到 node4  ,travel time: 10.0
在時刻 116 從node4 移動到 node7  ,travel time: 8.0
在時刻 124 從node7 移動到 node6  ,travel time: 5.0
在時刻 129 對node6進行保護  ,processing time: 10
在時刻 139 在node6 idle
在時刻 140 從node6 移動到 node7  ,travel time: 5.0
在時刻 145 對node7進行保護  ,processing time: 19
w:
w[6,1,139] 1.0
u:
u[1,38] 1.0
u[3,118] 1.0
u[5,38] 1.0
u[8,63] 1.0
u[9,114] 1.0
u[11,97] 1.0
u[12,103] 1.0
u[16,80] 1.0
u[17,120] 1.0
u[19,73] 1.0
u[20,45] 1.0
u[21,41] 1.0
u[22,0] 1.0
u[23,130] 1.0
u[24,91] 1.0
u[25,101] 1.0
u[26,140] 1.0
u[27,40] 1.0
u[29,97] 1.0
u[30,34] 1.0
u_bar:
u_bar[6,1,129] 1.0
u_bar[7,1,145] 1.0
u_bar[15,1,39] 1.0
u_bar[18,1,

v[21,108] 1.0
v[21,109] 1.0
v[21,110] 1.0
v[21,111] 1.0
v[21,112] 1.0
v[21,113] 1.0
v[21,114] 1.0
v[21,115] 1.0
v[21,116] 1.0
v[21,117] 1.0
v[21,118] 1.0
v[21,119] 1.0
v[21,120] 1.0
v[21,121] 1.0
v[21,122] 1.0
v[21,123] 1.0
v[21,124] 1.0
v[21,125] 1.0
v[21,126] 1.0
v[21,127] 1.0
v[21,128] 1.0
v[21,129] 1.0
v[21,130] 1.0
v[21,131] 1.0
v[21,132] 1.0
v[21,133] 1.0
v[21,134] 1.0
v[21,135] 1.0
v[21,136] 1.0
v[21,137] 1.0
v[21,138] 1.0
v[21,139] 1.0
v[21,140] 1.0
v[21,141] 1.0
v[21,142] 1.0
v[21,143] 1.0
v[21,144] 1.0
v[21,145] 1.0
v[21,146] 1.0
v[21,147] 1.0
v[21,148] 1.0
v[21,149] 1.0
v[21,150] 1.0
v[22,1] 1.0
v[22,2] 1.0
v[22,3] 1.0
v[22,4] 1.0
v[22,5] 1.0
v[22,6] 1.0
v[22,7] 1.0
v[22,8] 1.0
v[22,9] 1.0
v[22,10] 1.0
v[22,11] 1.0
v[22,12] 1.0
v[22,13] 1.0
v[22,14] 1.0
v[22,15] 1.0
v[22,16] 1.0
v[22,17] 1.0
v[22,18] 1.0
v[22,19] 1.0
v[22,20] 1.0
v[22,21] 1.0
v[22,22] 1.0
v[22,23] 1.0
v[22,24] 1.0
v[22,25] 1.0
v[22,26] 1.0
v[22,27] 1.0
v[22,28] 1.0
v[22,29] 1.0
v[22,30] 1.0
v[22,31] 1.0
v[22

v_bar[18,69] 1.0
v_bar[18,70] 1.0
v_bar[18,71] 1.0
v_bar[18,72] 1.0
v_bar[18,73] 1.0
v_bar[18,74] 1.0
v_bar[18,75] 1.0
v_bar[18,76] 1.0
v_bar[18,77] 1.0
v_bar[18,78] 1.0
v_bar[18,79] 1.0
v_bar[18,80] 1.0
v_bar[18,81] 1.0
v_bar[18,82] 1.0
v_bar[18,83] 1.0
v_bar[18,84] 1.0
v_bar[18,85] 1.0
v_bar[18,86] 1.0
v_bar[18,87] 1.0
v_bar[18,88] 1.0
v_bar[18,89] 1.0
v_bar[18,90] 1.0
v_bar[18,91] 1.0
v_bar[18,92] 1.0
v_bar[18,93] 1.0
v_bar[18,94] 1.0
v_bar[18,95] 1.0
v_bar[18,96] 1.0
v_bar[18,97] 1.0
v_bar[18,98] 1.0
v_bar[18,99] 1.0
v_bar[18,100] 1.0
v_bar[18,101] 1.0
v_bar[18,102] 1.0
v_bar[18,103] 1.0
v_bar[18,104] 1.0
v_bar[18,105] 1.0
v_bar[18,106] 1.0
v_bar[18,107] 1.0
v_bar[18,108] 1.0
v_bar[18,109] 1.0
v_bar[18,110] 1.0
v_bar[18,111] 1.0
v_bar[18,112] 1.0
v_bar[18,113] 1.0
v_bar[18,114] 1.0
v_bar[18,115] 1.0
v_bar[18,116] 1.0
v_bar[18,117] 1.0
v_bar[18,118] 1.0
v_bar[18,119] 1.0
v_bar[18,120] 1.0
v_bar[18,121] 1.0
v_bar[18,122] 1.0
v_bar[18,123] 1.0
v_bar[18,124] 1.0
v_bar[18,125] 1.0
v_bar